# Top Brass (Modular version) -- Example 5.1 in Rardin (1998)

Top Brass Trophy Company makes large championship trophies for youth athletic leagues. At the moment, they are planning production for fall sports: football and soccer. Each football trophy has a wood base, an engraved plaque, a large brass football on top, and returns \\$12 in profit. Soccer trophies are similar (a brass ball, a wood base, and an engraved plaque) except that a brass soccer ball is on top, and the unit profit is only \\$9. Since the football has an asymmetric shape, its base requires 4 board feet of wood; the soccer base requires only 2 board feet. At the moment there are 1000 brass footballs in stock, 1500 soccer balls, 1750 plaques, and 4800 board feet of wood. What trophies should be produced from these supplies to maximize total profit assuming that all that are made can be sold?

## Problem Data

We can improve our code's modifiability and reusability by separating the _data_ from the _structure_.

In [1]:
# An Array in Julia is a list of elements
trophy_types = [:football, :soccer] # these are the possible trophy types

# this array is of type "Symbol." Putting a colon in front of an element makes it a "Symbol" type.
# typically Symbol arrays are easier to implement in JuMP than other types (such as string, integer, etc.)

# A Dictionary in Julia maps a key (any type) to an element (any type)
wood_req = Dict(:football => 4, :soccer => 2) # how much wood each trophy type will use

# in this Dictionary, we are mapping keys of Symbol type (:football) to elements of integer type (4)

plaque_req = Dict(:football => 1, :soccer => 1) # how many plaques each trophy type will use

profit = Dict( :football => 12, :soccer => 9) # profit produced by each trophy type

# we are told the amount of each resource we have available
wood_avail = 4800
plaques_avail = 1750
football_avail = 1000
soccer_avail = 1500;     # semicolons supress output

## Top Brass Model

Now that we've defined our _data_ we can build a _general model_.

In [2]:
# always specify which packages you're going to use
using JuMP, Clp

#create a new model object, specifying the solver
m = Model(Clp.Optimizer)

# trophy variable object is now a Dictionary indexed over trophy types (elements are variables)
@variable(m, trophy[trophy_types] >= 0)

# maximize profit by summing (profit/trophy * # trophies) for each type
@objective(m, Max, sum(profit[i] * trophy[i] for i in trophy_types) )   

@constraint(m, sum(wood_req[i] * trophy[i] for i in trophy_types) <= wood_avail) # use only available wood
@constraint(m, sum(plaque_req[i] * trophy[i] for i in trophy_types) <= plaques_avail) # use only available plaques
@constraint(m, trophy[:football] <= football_avail)  # use only available brass footballs
@constraint(m, trophy[:soccer] <= soccer_avail)  # use only available brass soccer balls

status = optimize!(m) # solve instance of model

println(value.(trophy)) # print values of trophy variables 
# note the output is quite ugly. we'll see how to make it look nicer soon.
println("Total profit will be \$", objective_value(m))
println("We will use ", value(sum(wood_req[i] * trophy[i] for i in trophy_types) ), " board feet of wood")
println("We will use ", value(sum(plaque_req[i] * trophy[i] for i in trophy_types) ), " plaques")

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:football, :soccer]
And data, a 2-element Vector{Float64}:
  650.0
 1100.0
Total profit will be $17700.0
We will use 4800.0 board feet of wood
We will use 1750.0 plaques
Coin0506I Presolve 2 (-2) rows, 2 (0) columns and 4 (-2) elements
Clp0006I 0  Obj -0 Dual inf 20.999998 (2)
Clp0006I 2  Obj 17700
Clp0000I Optimal - objective value 17700
Coin0511I After Postsolve, objective 17700, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 17700 - 2 iterations time 0.002, Presolve 0.00


Another way to do the same thing using "expression" objects:

In [3]:
# create expressions for the total amount of wood and plaques used over all trophies
# syntax is (<model name>, <expression name>, <algebraic function>)
@expression(m, tot_wood, sum(wood_req[i] * trophy[i] for i in trophy_types) )
@expression(m, tot_plaques, sum(plaque_req[i] * trophy[i] for i in trophy_types) )
# create an expression for the total amount of profit over all trohpies
@expression(m, tot_profit, sum(profit[i] * trophy[i] for i in trophy_types) )

# constraints become much simpler
@constraint(m, tot_wood <= wood_avail)
@constraint(m, tot_plaques <= plaques_avail)
@constraint(m, trophy[:football] <= football_avail)
@constraint(m, trophy[:soccer] <= soccer_avail)

# the objective is also very simple
@objective(m, Max, tot_profit)

# solve the instance of the model 
status = optimize!(m)

# print interesting solution components, including expression values
println(value.(trophy))
println("Total profit will be \$", objective_value(m))
println("We will use ", value(tot_wood), " board feet of wood")
println("We will use ", value(tot_plaques), " plaques")

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:football, :soccer]
And data, a 2-element Vector{Float64}:
  650.0
 1100.0
Total profit will be $17700.0
We will use 4800.0 board feet of wood
We will use 1750.0 plaques
Coin0506I Presolve 2 (-6) rows, 2 (0) columns and 4 (-8) elements
Clp0006I 0  Obj -0 Dual inf 20.999998 (2)
Clp0006I 2  Obj 17700
Clp0000I Optimal - objective value 17700
Coin0511I After Postsolve, objective 17700, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 17700 - 2 iterations time 0.002, Presolve 0.00
